# Newton's Method

Have you ever wondered how computers or ordinary calculators determine the square root of a number? After all, most programming languages only provide the standard arithmatical operators like `+`, `-`, or `*`. In this tutorial, we learn how the underlying algorithm works behind the `math.sqrt` function in Python's Standard Library.

The mathematical grounding lies in Newton's Method, a numerical method to approximate the roots of continuous functions. Below is a lecture by MIT's professor of linear algebra [Gilbert Strang](http://math.mit.edu/~gs/) outlining the basic idea.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("U0xlKuFqCuI", width="80%")

To summarize, Newton's Method starts by guessing a root of a function and then improving that guess by linearly approximating the function. As we will see, all we need to supply is a Python function that returns the $y$ value of a mathematical function $y = f(x)$ for a given $x_0$ and another Python function that does the same for the derivative thereof. Newton's method can be regarded as a framework that takes our user-defined functions and finds  the roots of $f$, i.e., the set  $\{x \in \mathbb{R}: f(x) = 0 \}$. We build up the method step by step.

## Mathematical Functions & Derivatives

The following defines a helper function that plots several functions we pass to it. We need the third-party library [matplotlib](https://matplotlib.org/) which we can install with the command-line tool `pip`.

In [ ]:
! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Disregarding the default arguments (that only define the part of the $xy$-plane shown) we can call `plot()` like so: `plot(f1, f2, ...)`. `f1`, `f2`, and so on are references to function objects that expect a single argument `x` and return the corresponding value $y$.

In [ ]:
def plot(*functions, x_min=-3, x_max=3, y_min=-10, y_max=10, granularity=0.1, ax=None):
    n = int((x_max - x_min) / granularity) + 1
    xs = [x_min + x * granularity for x in range(n)]
    if ax:
        ax.clear()
    else:
        ax = plt.subplot()
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.plot(xs, [0 for x in xs], color="black")
    for f in functions:
        ax.plot(xs, [f(x) for x in xs]);

Let's use the following function $f_1$ as a simple example:

$f_1(x) = x^3 - x + 3$

We can translate it into Python as function `f1`.

In [ ]:
def f1(x):
    return ...  # put an arithmetic expression here

In [ ]:
plot(f1)

To obtain the derivative, we first need to do the work manually (computers can not do that by themselves):

$\frac{d}{dx} f_1(x) = f'_1(x) = 3x^2 - 1$

Then, we can also translate this into a Python function `df1`.

In [ ]:
def df1(x):
    return ...  # put an arithmetic expression here

In [ ]:
plot(f1, df1)

## Naïve Guessing

We can now guess where $f_1$ crosses $0$ by looking at the plot but this results in tedious work.

In [ ]:
f1(-2)

In [ ]:
f1(-1.5)

In [ ]:
f1(-1.7)

In [ ]:
f1(-1.67)

## Linear Approximations

Given a function and its derivative we can find an approximation for the $y$ value at an $x_1$ given an $x_0$ and the corresponding $y_0 = f(x_0)$ like so:

$y_1 \approx f(x_0) + f'(x_0) * (x_1 - x_0)$

Let's write a function that takes a mathematical function, its derivative, and $x_0$ and $x_1$ as arguments and returns an estimate for $y_1$ according to this rule.

In [ ]:
def approx(func, deriv, x0, x1):
    return func(x0) + ...  # complete the approximation with an arithmetic expression

`approx()` only returns an approximation for a single point $x_1$ and takes four arguments. For example, knowing that $f_1(-1) = 3$ and $f'_1(-1) = 2$, we can obtain an estimate for $x_1 = 0$.

In [ ]:
approx(f1, df1, -1, 0)

Such approximations of individual points do not seem of great value at first but visualizing the approximation rule as a **tangent line** at $x_0$ helps us understand how we can improve the approximations further below.

However, in order to plot a tangent line at $x_0$, we actually need some function that only expects a single `x` as its argument. Otherwise, `plot()` does not know how to use that function.

We use the following "trick" where an outer function `create_tangent()` takes the arguments `func`, `deriv`, and `x0` and return an inner and "dynamically" defined function `tangent()` that only has one argument `x` and that just returns the approximate value for that `x`. The latter can be used for plotting tangents.

When we refer to `func`, `deriv`, and `x0` in the inner function, Python remembers their values even after the call to `create_tangent()` is finished. The technical term for this is [closure](https://en.wikipedia.org/wiki/Closure_%28computer_programming%29) as the outer function "closes" the inner function.

In [ ]:
def create_tangent(func, deriv, x0):
    def tangent(x):
        return ...  # call the correct function with four arguments
    return tangent

In [ ]:
tan1 = create_tangent(f1, df1, -1)

`tan1` is really just a reference to a function object.

In [ ]:
tan1

In [ ]:
plot(f1, tan1)

## Educated Guessing

Just like before, we start with a somewhat "random" guess, like $x_0 = -1$. We refer to $x_0$ as `guess` in the following to avoid confusion with the `x0` arguments in some of the other functions.

In [ ]:
guess = -1

In [ ]:
f1(guess)

As we are interested in finding the roots of a function, we can plug in $0$ for $y_1$:

$0 \approx f(x_0) + f'(x_0) * (x_1 - x_0)$

Then, after rearranging the terms, we can find a better guess, by "optimizing" the $x_1$:

$x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$

Let's translate this into Python.

In [ ]:
def next_guess(func, deriv, x0):
    return ...  # put an arithmetic expression here

In [ ]:
next_guess(f1, df1, guess)

Let's capture these return values and then repeat the guessing process a couple of more times. $x_1$ then becomes $x_0$ in the next iteration. Also, we use `create_tangent()` and `plot()` to visualize the updated versions of the tangent line. For convenience, we compose these two functions into a new function `plot_tangent()`.

In [ ]:
def plot_tangent(func, deriv, x0, **kwargs):
    plot(func, create_tangent(func, deriv, x0), **kwargs)

Our random guess is quite far away from $0$.

In [ ]:
guess, f1(guess)

In [ ]:
plot_tangent(f1, df1, guess)

After the 1st iteration, we are actually further away from the root.

In [ ]:
guess = next_guess(f1, df1, guess)

In [ ]:
guess, f1(guess)

In [ ]:
plot_tangent(f1, df1, guess)

After the 2nd iteration, we seem to be getting where we want.

In [ ]:
guess = next_guess(f1, df1, guess)

In [ ]:
guess, f1(guess)

In [ ]:
plot_tangent(f1, df1, guess)

After the 3rd iteration, we have almost made it.

In [ ]:
guess = next_guess(f1, df1, guess)

In [ ]:
guess, f1(guess)

In [ ]:
plot_tangent(f1, df1, guess)

After the 4th iteration, we can hardly tell the difference from $0$.

In [ ]:
guess = next_guess(f1, df1, guess)

In [ ]:
guess, f1(guess)

In [ ]:
plot_tangent(f1, df1, guess)

## A first Search Algorithm

How can we tell if we have reached $0$ close enough? What happens if our initial random guess is way off? Does this procedure always terminate or might we end up in an infinite loop?

Let us define a `show_search()` function that takes the arguments `func`, `deriv`, `guess`, and `epsilon` and implements a the iterative search procedure with a `while` loop and shows the tangent's development over time. `epsilon` is the fault tolerance and defaults to $10^{-15}$.

In [ ]:
from IPython import display
from time import sleep

In [ ]:
def show_search(func, deriv, guess, epsilon=1e-15, **kwargs):

    # only for interactive plotting
    kwargs.update(ax=plt.subplot())
    n_iter = 0

    # Iterate until the guess is good enough
    while ... > epsilon:  # compare the absolute error to the allowed fault tolerance

        # only for interactive plotting
        print("Guess: {:30.20f}   (Iteration #{})".format(guess, n_iter))
        plot_tangent(f1, df1, guess, **kwargs)
        n_iter += 1
        display.display(plt.gcf())
        display.clear_output(wait=True)
        sleep(0.5)

        guess = ...  # call the correct function to obtain the next educated guess

    # only for interactive plotting
    print("Final: {:30.20f}   (Iteration #{})".format(guess, n_iter))

In [ ]:
show_search(f1, df1, -1, x_min=-5, x_max=3)

If the initial guess is way off, we can still be lucky and the search finishes fast.

In [ ]:
show_search(f1, df1, -1000, x_min=-10, x_max=3)

If our initial guess is near a flat part of $f$, the next guesses will "overshoot" the actual root by a lot and then has trouble converging.

In [ ]:
show_search(f1, df1, -0.577, x_min=-10, x_max=10)

## Local Search

Let us finalize our iterative search procedure with a function `local_search()` that takes the same arguments as `show_search()` and a `max_iter` argument defaulting to $1000$ that prevents the search to go into an infinite loop. The function either returns a tuple of the root it found and the number of iterations or raises a `RuntimeError` if it did not converge.

In [ ]:
def local_search(func, deriv, guess, epsilon=1e-15, max_iter=1000):
    n_iter = ...  # track the number of iterations (1)

    # Iterate until the guess is good enough
    # or the maximum number of iterations are reached
    while (...) and (...):  # copy the stopping criterion from show_search()
                            # and add a check that no more than max_iter iterations can happen

        ...  # update the number of iterations (2)

        ...  # update the current best guess

    # If the search procedure did not converge
    if n_iter == max_iter:
        raise RuntimeError("Local search did not converge!")

    return guess, ...  # complete the return value according to the description above

Newton's Method is very robust with respect to the initial guess.

In [ ]:
local_search(f1, df1, -1)

In [ ]:
local_search(f1, df1, -1_000)

In [ ]:
local_search(f1, df1, -1_000_000)

In [ ]:
local_search(f1, df1, -1_000_000_000)

In [ ]:
local_search(f1, df1, -1_000_000_000_000)

By allowing less fault tolerance `epsilon`, the search procedure does not converge.

In [ ]:
local_search(f1, df1, -1_000_000_000_000, epsilon=1e-20)

Even starting around flat line segments works.

In [ ]:
local_search(f1, df1, -0.5773)

In [ ]:
local_search(f1, df1, -0.577350)

In [ ]:
local_search(f1, df1, -0.57735026)

In [ ]:
local_search(f1, df1, -0.5773502691)

In [ ]:
local_search(f1, df1, -0.577350269189)

However, plugging in the exact $x$ value of a local maximum or minimum causes troubles. Here, we use $x_0 = -\sqrt{\frac{1}{3}}$.

In [ ]:
local_search(f1, df1, -(1/3) ** (1/2)) 

We can avoid a crashing `local_search()` by wrapping the function with another function `search()` that handles exceptions and re-calls `local_search()` repeatedly with a slightly different initial guess until the search converges. To avoid the possibility of another infinite loop we restrict this to `max_retries` tries defaulting to $10$.

In [ ]:
from random import random

In [ ]:
def search(func, deriv, guess, epsilon=1e-15, max_iter=1000, max_retries=10):
    for _ in range(max_retries):
        try:
            root, _ = ...  # call local_search() with the correct arguments
        except ZeroDivisionError:
            guess += random()
        else:
            return root
    else:
        raise RuntimeError("Local search did not converge repeatedly!")

Now, Newton's Method even works with $x_0 = -\sqrt{\frac{1}{3}}$ as the initial guess.

In [ ]:
search(f1, df1, -(1/3) ** (1/2))

## Square Roots

Finally, we implement our own `sqrt()` function that takes one mandatory argument `n` and an optional argument `guess` (defaulting to $0$) and calculates the square root by calling `search()` with the correct `func` and `deriv` arguments.

What are these correct arguments? Observe that a function $f_2(x) = x^2 - n$ has its roots where $x^2 = n$ and its derivative is $f'_2(x) = 2x$. That implies that $f_2$ depends on the number `n` that we want to find the square root of and we need to revert to the same trick as above with `create_tangent()` and use an outer function `create_f2()` that dynamically creates the `f2()` functions.

In [ ]:
def create_f2(n):
    def f2(x):
        return ...  # put in an arithmetic expression
    return f2

In [ ]:
def df2(x):
    return ...  # put in an arithmetic expression

In [ ]:
def sqrt(n, guess=0):
    f2 = ...  # call create_f2() with the correct argument
    return search(f2, df2, guess)

Now we can calculate square roots without the `math` module in the Standard Library.

In [ ]:
sqrt(2)

In [ ]:
sqrt(9)

Alternatively, we can use two `lambda` expressions and formulate `sqrt()` compactly.

In [ ]:
def sqrt(n, guess=0):
    return search(lambda x: ..., lambda x: ..., guess)  # put in the two arithmetic expressions from before

In [ ]:
sqrt(2)

In [ ]:
sqrt(9)